In [1]:
%matplotlib inline
import os
import sys


import scipy as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as multi


import csv
import collections as col


import statsmodels.stats.multitest as multi

# Beta binomial testing for changes in cell state abundance

**Model:** 
* A fraction _f_ of cells in a given condition are in state X; 
* The fraction _f_ is beta-distributed with parameters a,b: _f~B(a,b)_ 
* In a given experiment, a total of _N_ cells are sampled and _n_ are found in state X
* For two conditions, the null hypothesis is that underlying beta distribution has same (a,b).
* The alternative is that (a,b) are changed.

The test is implemented for unpaired and paired samples in the papers:
* Pham TV, Piersma SR, Warmoes M, Jimenez CR (2010) On the beta binomial model for analysis of spectral count data in label-free tandem mass spectrometry-based proteomics. Bioinformatics, 26(3):363-369.
* Pham TV, Jimenez CR (2012) An accurate paired sample test for count data. Bioinformatics, 28(18):i596-i602.

The authors of these papers have written code in R to implement the tests. The code is available here:
https://tvpham.github.io/ibb/

** Comments on implementation:** 
I struggled to get the ibb code to work using my miniconda-installed R. I managed to get it to work in an R studio, with R version 4.0.

Because of this, I am not using Rpy2 to call ibb directly from Python, sorry. I am generating an R script to run in R studio...

## Load the classified cell counts

In [2]:
# The following files have a first column giving the state name, 
#  and subsequent columns have cell counts per replicate. 
# Data is paired, so that first count column is from matched ctrl and perturbed.

study_name = 'HUA vs ctrl' # Used for plotting only
raw1 = pd.read_csv('../../../data/abundance_data/abundances_hua_ctrl_v3p2.csv',index_col=0, sep=r'\s*,\s*', encoding='ascii', engine='python')
raw2 = pd.read_csv('../../../data/abundance_data/abundances_hua_perturbed_v3p2.csv', index_col=0, sep=r'\s*,\s*', encoding='ascii', engine='python')

group_name1 = 'Ctrl'
group_name2 = 'HUA'

## Prepare data for testing:

In [3]:
# Calculate total counts per experiment:
totals1 = raw1.sum(axis=0)
totals2 = raw2.sum(axis=0)

In [4]:
# Prepare groups string for R:
hdrStr = 'pVals = c()'
groupStr = 'group <- c(rep("' + group_name1 + '", ' \
    + str(len(totals1))  + '), rep("' + group_name2 + '", ' \
    + str(len(totals2))  + '))'

# Prepare total counts string for R:
tmpStr1 = [str(x) for x in np.array(totals1)]
tmpStr2 = [str(x) for x in np.array(totals2)]
totalsStr = 'tx <- c(' + (",".join(tmpStr1)) +',' + (",".join(tmpStr2)) + ')'

# Prepare state counts strings for R:
xStr = {}
for state in raw1.index:
    tmpStr1 = [str(x) for x in np.array(raw1.loc[state])]
    tmpStr2 = [str(x) for x in np.array(raw2.loc[state])]
    xStr[state] = 'x <- c(' + ",".join(tmpStr1) +',' + ",".join(tmpStr2) + ')'


# Prepare call to ibb.test (paired data testing) for R:
ibbStr = 'ibbOut <- ibb.test(x, tx, group)'

# Save:
saveStr = 'pVals <- c(pVals, ibbOut$p.value)'
printStr = 'cat(pVals,sep=",")' # Prints the p values as a single comma-separated list


## Run the tests:

### Generate a file to run in R studio:

In [5]:
ibb_script_file = 'R_scripts_for_ibb/R_ibb_script_hua_states_v3p2.txt'

out_file = open(ibb_script_file, "w")

out_file.write(hdrStr+'\n')
out_file.write(groupStr+'\n')
out_file.write(totalsStr+'\n')

for state in raw1.index:
    out_file.write(xStr[state]+'\n')
    out_file.write(ibbStr+'\n')
    out_file.write(saveStr+'\n')

    
out_file.write(printStr+'\n')
out_file.close()    

### RUN THE FILE IN R Studio AND THEN COPY RESULTS HERE

In [6]:
# Paste the comma-separated list of results into pVals = [...]
#pVals = [0.0002193295,3.14742e-05,0.0007482395,0.06476054,0.001943749,0.05277143,0.0002072166,0.06339063,4.345061e-05,0.01674463,0.005910348,0.0002363588,8.900534e-06,0.0001707545,0.1591516,0.7880153,0.007401928,0.1272023,0.0009984922,0.1470245,4.70061e-07,8.719192e-11,0.001828684,2.392008e-13,0.0002435873,0.226999,0.001765101,0.02193535,0.002713158,0.6551954,0.001527332,0.3737026,0.01131933,0.1621698,0.0575569,0.008208384,0.000470411,0.1498871,0.7473183]
#pVals = [0.0002048317,4.23047e-05,0.0009531208,0.1145396,0.002736659,0.06283012,0.0001799602,0.07274826,1.518206e-05,0.0117441,0.009987051,0.0001446854,2.303698e-06,8.948266e-05,0.1638035,0.4109072,0.01494496,0.1286229,0.001480217,0.1306519,2.108508e-07,8.093966e-12,0.0007102856,3.222851e-11,0.0002435873,0.2198374,0.003784941,0.02351056,0.001359104,0.5891708,0.001943994,0.3792333,0.003698796,0.2108631,0.2288228,0.1725085,0.007142845,0.05529975,0.5364335]
#pVals = [0.0002048317,4.23047e-05,0.0009531208,0.1145396,0.002736659,0.06283012,0.0001799602,0.07274826,1.518206e-05,0.0117441,0.009987051,0.0001446854,2.303698e-06,8.948266e-05,0.1638035,0.4109072,0.01494496,0.1286229,0.001480217,0.1306519,2.108508e-07,0.0215349,0.0002435873,0.2198374,0.003784941,0.02351056,0.001359104,0.5891708,0.001943994,0.3792333,0.003698796,0.2108631,0.2288228,0.1725085,0.007142845,0.05529975,0.5364335]

pVals = [0.0002177271,4.057692e-05,0.0007904614,0.1213262,0.004097619,0.2058659,0.0001547909,0.06708271,1.437957e-05,0.01246733,0.0125549,0.00329251,8.085408e-06,0.0001976922,0.1421056,0.004912005,0.01016957,0.1328313,0.001461029,0.07400285,4.439291e-07,0.02461386,0.0004719933,0.07283528,0.003120756,0.02589988,0.001465,0.4911738,0.001398899,0.4173868,0.008377568,0.04974562,0.05928009,0.005207306,0.003576422,0.09787039,0.9654831]
pVals_df = pd.DataFrame(index= raw1.index)
pVals_df['ibb_p'] = pVals


bh_out = multi.multipletests(pVals_df['ibb_p'], alpha=0.05, method='fdr_bh')
pVals_df['ibb_p_corr'] = bh_out[1]


In [7]:
pVals_df.to_csv('../../../data/abundance_data/23-01-15_ibb_pvals_hua_states_v3p2.csv')